<hr>

Let's talk about ***Decorators***.

<hr>

In [70]:
from functools import (
    wraps, update_wrapper, 
    lru_cache, 
    total_ordering
)

from pprint import pprint, pformat

0x01 - the ```@wraps```

In [71]:
def eggs(func):
    
    @wraps(func)  # crucial line for deco (preserve the func's state)
    def _eggs(*args, **kwargs):
        ''' doc for deco '''
        print(f'{func.__name__!r} got args: {args!r} and ..')
        return func(*args, **kwargs)

    return _eggs

In [72]:
@eggs
def spam(a, b, c):
    ''' doc for spam '''
    return a * b * c

# its name, classname, ... are all preserved (by @wraps)
spam.__name__
help(spam)

'spam'

Help on function spam in module __main__:

spam(a, b, c)
    doc for spam



0x02 - *debug*

In [73]:
def debug(func):
    
    @wraps(func)
    def _debug(*args, **kwargs):
        ''' almost the same as the first example XD '''
        
        output = func(*args, **kwargs)
        print(f'{func.__name__!r} got args: {args!r}')
        
        return output
    return _debug 


@debug
def sayhi(times):
    return 'Hi ' * times

sayhi(5)

'sayhi' got args: (5,)


'Hi Hi Hi Hi Hi '

0x03 - *caching*

In [74]:
def memoize(func):
    ''' 
    a similar func `lru_cache` introduce by Python in ver3.2 
    
    It (lru_cache) 
      contains a fixed cache size (128 by default) to save the memory
      and use some stat to check whether the size should be increased
    '''
    
    func.cache = dict()  # access it by '__wrapped__.cache' in outer scope 
    
    @wraps(func)
    def _memoize(*args):
        
        if args not in func.cache:
            func.cache[args] = func(*args)
        
        return func.cache[args]
    return _memoize

In [75]:
# example: Fib

@memoize
def fib(n):
    if n < 2:
        return n
    else:
        return fib(n - 1) + fib(n - 2)
    
for i in range(1, 5):
    i, fib(i)
    
# check out the cache!
fib.__wrapped__.cache

(1, 1)

(2, 1)

(3, 2)

(4, 3)

{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3}

In [76]:
# example: how `lru_cache` works internally 

def counter(func):
    
    func.calls = 0
    
    @wraps(func)
    def _counter(*args, **kwargs):
        
        func.calls += 1
        
        return func(*args, **kwargs)
    return _counter


@lru_cache(maxsize=3)
@counter
def fib(n):
    if n < 2:
        return n 
    else:
        return fib(n - 1) + fib(n - 2)

In [77]:
fib(100)          # other num is fine as well :)
fib.cache_info()  # sort of statistics of caching 

fib.__wrapped__.__wrapped__.calls 

354224848179261915075

CacheInfo(hits=98, misses=101, maxsize=3, currsize=3)

101

0x04 - deco with (optional) args

In [78]:
# well

def add(extra_n=1):
    ''' nesting! '''
    
    def _add(func):  
        ''' here's the real deco '''
        
        @wraps(func)
        def __add(n):
            return func(n + extra_n)  # e.g. sayhi(2+3)
        
        return __add
    return _add


# actual calls 
#   { the result might be changed by the decorated version (sayhi) }
add(extra_n=2)(sayhi)(2)
add(sayhi)(2)

'sayhi' got args: (4,)


'Hi Hi Hi Hi '

<function __main__.add.<locals>._add.<locals>.__add>

In [79]:
# example: call with decorators 

@add(extra_n=2)
def sayhi(n):
    return 'hi ' * n

sayhi(3)

'hi hi hi hi hi '

In [80]:
# example: 
#   check the deco was called with a function OR a regular param

def add(*args, **kwargs):
    ''' add n to the input of the decorated func 
    '''
    
    default_kwargs = dict(n=1)
    
    def _add(func):
        
        @wraps(func)
        def __add(n):
            default_kwargs.update(kwargs)
            
            return func(n + default_kwargs['n'])
        return __add
    
    if len(args) == 1 and callable(args[0]) and not kwargs: 
        
        return _add(args[0])
    
    elif not args and kwargs:    
        
        default_kwargs.update(kwargs)
        return _add
    
    else:
        raise RuntimeError('This deco only supports kw args.')

In [81]:
# ---- no arg ---- 

@add
def hi(n):
    return 'Hi ' * n

hi(3)


# ---- with kw args ---- 
    
@add(n=3) 
def yo(n):
    return 'Yo ' * n

yo(3)

'Hi Hi Hi Hi '

'Yo Yo Yo Yo Yo Yo '

0x05 - creating deco using classes

In [82]:
# take `Debug` as an example 

class Debug(object):
    ''' by using classes, 
          ya need change the `wraps` to `update_wrapper` !! 
    '''
    
    def __init__(self, func):
        
        self.func = func 
        update_wrapper(self, func)
        
    def __call__(self, *args, **kwargs):
        
        output = self.func(*args, **kwargs)
        print(f'{self.func.__name__} --- {args}, {kwargs} --- {output!r}')
        
        return output

In [83]:
@Debug
def sayhi(n):
    return 'Hi' * n

sayhi(5)


output = sayhi(3)
output

sayhi --- (5,), {} --- 'HiHiHiHiHi'


'HiHiHiHiHi'

sayhi --- (3,), {} --- 'HiHiHi'


'HiHiHi'

0x06 - decorating ***class functions***

In [84]:
# example: aha

def plus_one(func):
    
    @wraps(func)
    def _plus_one(self, n):
        return func(self, n + 1)
    
    return _plus_one


class SayIt(object):
    @plus_one
    def hi(self, n=2):
        return 'Hi' * n

    
sayit = SayIt()
sayit.hi(2)

'HiHiHi'

In [85]:
# example: `classmethod` & `staticmethod

class Greet(object):
    
    def some_instmethod(self, *args, **kwargs):
        print(f'self   :  {self!r}')
        print(f'args   :  {pformat(args)}')   # `pformat` from 'pprint'
        print(f'kwargs :  {pformat(kwargs)}') 
        
    @classmethod
    def some_clsmethod(cls, *args, **kwargs):
        print(f'cls    :  {cls!r}')
        print(f'args   :  {pformat(args)}')    
        print(f'kwargs :  {pformat(kwargs)}')  
        
    @staticmethod
    def some_statmethod(self, *args, **kwargs):
        print(f'args   :  {pformat(args)}')    
        print(f'kwargs :  {pformat(kwargs)}')  

        
hi = Greet()

In [86]:
# partOne: inst only

# same as usual ( 1,2 for arg | 3,4 for kwargs )
hi.some_instmethod(1, 2, a=3, b=4)         


try:
    Greet.some_instmethod()                # ya need the `self`

except Exception as err:
    err
    
finally:
    Greet.some_instmethod(1, 2, a=3, b=4)  # '1' as `self` (incorrectly!!)

self   :  <__main__.Greet object at 0x10f105c18>
args   :  (1, 2)
kwargs :  {'a': 3, 'b': 4}


TypeError("some_instmethod() missing 1 required positional argument: 'self'")

self   :  1
args   :  (2,)
kwargs :  {'a': 3, 'b': 4}


In [87]:
# partTwo: cls only 

hi.some_clsmethod(1, 2, a=3, b=4)

print(); Greet.some_clsmethod()                # `None`, but no error :)
print(); Greet.some_clsmethod(1, 2, a=3, b=4)  # same as usual (correctly)

cls    :  <class '__main__.Greet'>
args   :  (1, 2)
kwargs :  {'a': 3, 'b': 4}

cls    :  <class '__main__.Greet'>
args   :  ()
kwargs :  {}

cls    :  <class '__main__.Greet'>
args   :  (1, 2)
kwargs :  {'a': 3, 'b': 4}


In [88]:
# partThr: static only 

hi.some_statmethod(1, 2, a=3, b=4)


try:
    Greet.some_statmethod()
    
except Exception as err:
    err
    
finally:
    Greet.some_statmethod(1, 2, a=3, b=4)

args   :  (2,)
kwargs :  {'a': 3, 'b': 4}


TypeError("some_statmethod() missing 1 required positional argument: 'self'")

args   :  (2,)
kwargs :  {'a': 3, 'b': 4}


0x07 -  decorating ***class functions***" using ```descriptor```

In [89]:
class MoreSpam(object):
    
    def __init__(self, more=1):
        self.more = more 
        
    def __get__(self, inst, cls):
        return self.more + inst.spam
    
    def __set__(self, inst, val):
        inst.spam = val - self.more 
        
class Spam(object):
    
    more_spam = MoreSpam(5)
    
    def __init__(self, spam):
        self.spam = spam  
        

e = Spam(2)

e.spam, '->', e.more_spam

e.more_spam = 11

e.spam, '->', e.more_spam

(2, '->', 7)

(6, '->', 11)

0x08 - more about "decorating ***class functions***"

In [90]:
class ClassMethod(object):
    
    def __init__(self, method):
        self.method = method
        
    def __get__(self, inst, cls):
        
        @wraps(self.method)
        def method(*args, **kwargs):
            return self.method(cls, *args, **kwargs)
        
        return method
    
    
class StaticMethod(object):
    
    def __init__(self, method):
        self.method = method
        
    def __get__(self, inst, cls):
        return self.method

0x09 - ```property```

In [91]:
class GreetOne(object):
    ''' one way for using `property` decorator '''
    
    def get_amount(self):
        print('gettin amount')
        return self._amount
    
    def set_amount(self, amount):
        print(f'setting amount to {amount}')
        self._amount = amount
        
    def del_amount(self):
        print('deletin amount')
        del self._amount
        
    amount = property(get_amount, set_amount, del_amount)
    

class GreetTwo(object):
    ''' the other way using `property` decorator '''
    
    @property
    def amount(self):
        print('gettin amount')
        return self._amount
    
    @amount.setter
    def amount(self, amount):
        print(f'settin amount to {amount}')
        self._amount = amount
        
    @amount.deleter
    def amount(self):
        print('deletin amount')

In [92]:
def hi_one():

    hi = GreetOne()
    
    hi.amount = 5
    hi.amount
    
    del hi.amount
    
def hi_two():
    
    hi = GreetTwo()
    
    hi.amount = 10
    hi.amount 
    
    del hi.amount
    
     
hi_one(); print()
hi_two()

setting amount to 5
gettin amount
deletin amount

settin amount to 10
gettin amount
deletin amount


0x10 - implementation of ```property``` (roughly)

In [93]:
# the implementation of `Property`  (well, by author)


class Property(object):
    ''' TODO: understand this XD '''

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel

        if fget and not doc:
            doc = fget.__doc__

        self.__doc__ = doc

    def __get__(self, inst, cls):
        if inst is None:
            return self
        elif self.fget:
            return self.fget(inst)
        else:
            raise AttributeError('unreadale attr')

    def __set__(self, inst, val):
        if self.fset:
            self.fset(inst, val)
        else:
            raise AttributeError('cannot set attr')

    def __delete__(self, inst):
        if self.fdel:
            self.fdel(inst)
        else:
            raise AttributeError('cannot del attr')

    def getter(self, fget):
        return type(self)(
            fget,
            self.fset,
            self.fdel
        )

    def setter(self, fget):
        return type(self)(
            self.fget,
            fset,
            self.fdel
        )

    def deleter(self, fget):
        return type(self)(
            self.fget,
            self.fset,
            fdel
        )

0x11 - the ```__getattr__``` & else

In [94]:
class Spam(object):
    
    def __init__(self):
        self.registry = {}
        
    def __getattr__(self, key):
        print(f'gettin {key!r}')
        return self.registry.get(key, 'Undefined')
    
    def __setattr__(self, key, val):
        if key == 'registry':
            object.__setattr__(self, key, val)
        else:
            print(f'settin {key!r} to {val!r}')
            self.registry[key] = val 
            
    def __delattr__(self, key):
        print(f'deletin {key!r}') 
        

sp = Spam() 

sp.a    
sp.a = 1  
sp.a 

del sp.a 

gettin 'a'


'Undefined'

settin 'a' to 1
gettin 'a'


1

deletin 'a'


0x12 - decorating ***classes***

In [95]:
def singleton(cls):

    inst = dict()

    @wraps(cls)
    def _singleton(*args, **kwargs):

        if cls not in inst:
            inst[cls] = cls(*args, **kwargs)

        return inst[cls]
    return _singleton


@singleton
class Spam(object):
    def __init__(self):
        print('Excuting init...')

In [96]:
a = Spam() 
b = Spam()

a is b     # same object (truly)

a.ss = 10  # b also equals to 10
b.ss

Spam.__wrapped__

Excuting init...


True

10

__main__.Spam

0x13 - total **ordering** (for class)

In [97]:
class Value(object):
    ''' bases of all examples '''
    
    def __init__(self, val):
        self.val = val 
        
    def __repr__(self):
        return f'--- {self.__class__}[{self.val}] ---'
    
    
# method One: regular ways 

class Spam(Value):
    
    def __gt__(self, other):
        return self.val > other.val  
    
    def __ge__(self, other):
        return self.val >= other.val   
    
    def __lt__(self, other):
        return self.val < other.val   
    
    def __le__(self, other):
        return self.val <= other.val   
    
    def __eq__(self, other):
        return self.val == other.val 
    
    
# method Two: `total_ordering` ways 

@total_ordering
class Eggs(Value):
    def __lt__(self, other):
        return self.val < other.val 
    
    def __eq__(self, other):
        return self.val == other.val

In [98]:
# example: 

nums = [4, 1, 3, 2]

spam = [ Spam(n) for n in nums ]
eggs = [ Eggs(n) for n in nums ]


spam
eggs 

sorted(spam)
sorted(eggs)

[--- <class '__main__.Spam'>[4] ---,
 --- <class '__main__.Spam'>[1] ---,
 --- <class '__main__.Spam'>[3] ---,
 --- <class '__main__.Spam'>[2] ---]

[--- <class '__main__.Eggs'>[4] ---,
 --- <class '__main__.Eggs'>[1] ---,
 --- <class '__main__.Eggs'>[3] ---,
 --- <class '__main__.Eggs'>[2] ---]

[--- <class '__main__.Spam'>[1] ---,
 --- <class '__main__.Spam'>[2] ---,
 --- <class '__main__.Spam'>[3] ---,
 --- <class '__main__.Spam'>[4] ---]

[--- <class '__main__.Eggs'>[1] ---,
 --- <class '__main__.Eggs'>[2] ---,
 --- <class '__main__.Eggs'>[3] ---,
 --- <class '__main__.Eggs'>[4] ---]

In [99]:
# example: sort by key 

val = [Value(n) for n in nums]
val 

sorted(val, key=lambda v: v.val)

[--- <class '__main__.Value'>[4] ---,
 --- <class '__main__.Value'>[1] ---,
 --- <class '__main__.Value'>[3] ---,
 --- <class '__main__.Value'>[2] ---]

[--- <class '__main__.Value'>[1] ---,
 --- <class '__main__.Value'>[2] ---,
 --- <class '__main__.Value'>[3] ---,
 --- <class '__main__.Value'>[4] ---]

0x14 - DIY a decorator which '***make class sortable by specific key***'

In [100]:
class Value(object):
    ''' base class we're gonna use '''
    
    def __init__(self, val):
        self.val = val 
    
    def __repr__(self):
        return f'--- {self.__class__}[{self.val}] ---'
    
    
def sort_by_attr(attr, keyfunc=getattr):
    ''' the decorator! '''
    
    def _sort_by_attr(cls):
        
        def __gt__(self, other):
            return getattr(self, attr) > getattr(other, attr)
        
        def __ge__(self, other):
            return getattr(self, attr) >= getattr(other, attr)    
    
        def __lt__(self, other):
            return getattr(self, attr) < getattr(other, attr)

        def __le__(self, other):
            return getattr(self, attr) <= getattr(other, attr)

        def __eq__(self, other):
            return getattr(self, attr) == getattr(other, attr)
        
        
        cls.__gt__ = __gt__
        cls.__ge__ = __ge__
        cls.__lt__ = __lt__
        cls.__le__ = __le__
        cls.__eq__ = __eq__
        
        
        return cls 
    
    return _sort_by_attr

In [101]:
# example: showing how it works 

nums = [4, 2, 3, 2]


@sort_by_attr('val')
class Spam(Value):
    pass 


spams = [ Spam(n) for n in nums ]

sorted(spams)

[--- <class '__main__.Spam'>[2] ---,
 --- <class '__main__.Spam'>[2] ---,
 --- <class '__main__.Spam'>[3] ---,
 --- <class '__main__.Spam'>[4] ---]

<hr>

Now the ***advance part begins***.

<hr>

0x01 - *Polymorphism* <small>(多态)</small> in Python
- single dispatch pattern ( *I dunno what does that mean* )

In [102]:
# example: different data => different result (well, Polymorphism)

from functools import singledispatch


@singledispatch
def printer(val):
    ''' don't touch this (original) function '''
    
    print('else\t---- {!r}'.format(val))

    
@printer.register(str)
def str_printer(val):
    
    print('str\t---- {}'.format(val))

    
@printer.register(int)
def int_printer(val):
    
    print('int\t---- {:d}'.format(val))

    
@printer.register(dict)
def dict_printer(val):
    
    for k, v in sorted(val.items()):
        print(f'dict\t---- key: {k!r}, value: {v!r}') 

In [103]:
# nice! XD

printer(['this', 'is', 'fun'])

printer('hell yeah')

printer(666)

printer({'hmm':'I agree'})

else	---- ['this', 'is', 'fun']
str	---- hell yeah
int	---- 666
dict	---- key: 'hmm', value: 'I agree'


In [104]:
# example: play with filename & file handler

import json 
from functools import singledispatch


@singledispatch  
def write_as_json(file, data):
    json.dump(data, file)
    
@write_as_json.register(str)
@write_as_json.register(bytes)
def write_as_json_filename(file, data):
    
    with open(file, 'w') as fh:
        write_as_json(fh, data)

In [105]:
data = dict(a=1, b=2, c=3)

write_as_json('./src/test_01.json', data)

write_as_json(b'./src/test_02.json', data)

with open('./src/test_03.json', 'w') as fh:
    write_as_json(fh, data)
    
# check the file 
!ls -l src
!echo "\n"

# check the registered types 
for i, j in write_as_json.registry.items():
    print(f'{i}\n\t{j}')

total 1280
-rwxrwxrwx@ 1 alex  staff  24 Aug  9 23:04 test_01.json
-rwxrwxrwx@ 1 alex  staff  24 Aug  9 23:04 test_02.json
-rwxrwxrwx@ 1 alex  staff  24 Aug  9 23:04 test_03.json
-rwxrwxrwx  1 alex  staff  19 Aug  9 22:39 test_context_01.txt
-rwxrwxrwx  1 alex  staff  28 Aug  9 22:39 test_context_02.txt


<class 'object'>
	<function write_as_json at 0x10f146e18>
<class 'bytes'>
	<function write_as_json_filename at 0x10efd9048>
<class 'str'>
	<function write_as_json_filename at 0x10efd9048>


0x02 - let's talk about '*contextlib*' module

In [106]:
import contextlib

# example: build our own context manager (`open`)
#   (well, in this case, suppose we don't have the `with open` stuff!)

@contextlib.contextmanager
def open_cont_manager(filename, mode='r'):
    
    fh = open(filename, mode)
    
    yield fh 
    fh.close()

In [107]:
# simple XD
with open_cont_manager('./src/test_context_01.txt', 'w') as fh:
    print('contextlib is fun!', file=fh)
    
    
# yet another one 
with contextlib.closing(open('./src/test_context_02.txt', 'w')) as fh:
    print('well, this is gonna be fun!', file=fh)
    

!ls -l ./src/*context*

-rwxrwxrwx  1 alex  staff  19 Aug  9 23:04 ./src/test_context_01.txt
-rwxrwxrwx  1 alex  staff  28 Aug  9 23:04 ./src/test_context_02.txt


In [108]:
# example: use contextlib in other fields (more practical)

well = '''
    for file         ---- there's already a `with open`. 
    for some objects ---- don't support automatic closing (e.g. urllib)
'''

@contextlib.contextmanager
def debug(name):
    print(f'Debugging Start {name!r}')
    yield 
    print(f'Debugging End   {name!r}')
    

@debug('hello')
def hello():
    print("I'm in `hello()`")
    
hello()

Debugging Start 'hello'
I'm in `hello()`
Debugging End   'hello'


0x03 - *validation*, *type check*, and *conv* ...

In [109]:
# example: type hinting 

def sayhi(amount: int):    # right here!
    print('hi ' * amount)  
    
sayhi(5)

hi hi hi hi hi 


In [110]:
# example: type check & conv 

import inspect
from functools import wraps 


def to_int(name, mini=None, maxi=None):
    
    def _to_int(func):
        
        signature = inspect.signature(func)
        
        @wraps(func, ['__signature__'])
        @wraps(func)
        def __to_int(*args, **kwargs):
            
            bound   = signature.bind(*args, **kwargs)
            
            default = signature.parameters[name].default
            val     = int(bound.arguments.get(name, default))
            
            
            if mini is not None:
                
                assert val >= mini, (
                    f'{name} should be at least {mini!r}, got: {val!r}'
                )
                
            if maxi is not None:
                
                assert val <= maxi, (
                    f'{name} should be at most  {mini!r}, got: {val!r}'
                )
                
            return func(*args, **kwargs)
        
        return __to_int
    return _to_int

In [111]:
@to_int('a', mini=10)
@to_int('b', maxi=10)
@to_int('c')

def foo(a, b, c=10):
    
    print('a', a)
    print('b', b)
    print('c', c)
    
    
foo(10,   b=0)  ; print()
foo(a=20, b=10) ; print()


try:
    foo(1, 2, 3)                  # limitations ( by our 'mini | maxi' )
except AssertionError as err:
    err
    
    try:
        foo()                     # no default value ( well, should be )
    except TypeError as err:
        err
        
        try:
            foo('what', {})       # not convertable ( there's `int()` )
        except ValueError as err:  
            err

a 10
b 0
c 10

a 20
b 10
c 10



AssertionError('a should be at least 10, got: 1')

TypeError("missing a required argument: 'a'")

ValueError("invalid literal for int() with base 10: 'what'")

In [112]:
# example: type check & conv (not using `inspect`)

from functools import wraps 

def to_integer(name, mini=None, maxi=None):
    ''' this won't be as good as the previous `to_int()` decorator! '''
    
    def _to_integer(func): 
        
        @wraps(func) 
        def __to_integer(**kwargs):
            val = int(kwargs.get(name))
            
            if mini is not None:
                
                assert val >= mini, (
                    f'{name} should at at least {mini!r}, got: {val!r}'
                )
                
            if maxi is not None:
                
                assert val <= maxi, (
                    f'{name} should be at most  {maxi!r}, got: {val!r}'
                ) 
                
            return func(**kwargs)
        
        return __to_integer
    return _to_integer

In [113]:
@to_integer('a', mini=10)
@to_integer('b', maxi=10)
def bar(a, b):
    print('a', a)
    print('b', b)
    
    
bar(a=20, b=10)

try:
    bar(a=1, b=10)             # 1 < 10 (mini)
except AssertionError as err:
    err 

a 20
b 10


AssertionError('a should at at least 10, got: 1')

0x04 - ignore useless warnings

In [114]:
from functools import wraps
import warnings  


def ign_warning(warning, count=None):
    
    def _ign_warning(func): 
        
        @wraps(func)
        def __ign_warning(*args, **kwargs):
            
            with warnings.catch_warnings(record=True) as ws:
                
                warnings.simplefilter('always', warning)
                
                result = func(*args, **kwargs)
                
            
            if count is not None:
                
                for w in ws[count:]:
                    
                    warnings.showwarning(
                        message  = w.message,
                        category = w.category,
                        filename = w.filename,
                        lineno   = w.lineno,
                        file     = w.file,
                        line     = w.line,
                    )
            
            return result
        
        return __ign_warning
    return _ign_warning

In [115]:
@ign_warning(DeprecationWarning, count=1)
def spam():
    warnings.warn('\n u are weeeeeak (001)\n', DeprecationWarning)
    warnings.warn('\n\n catch me~~ (002)\n\n', DeprecationWarning)   
    warnings.warn('\n\n you got me (003)\n\n', DeprecationWarning)    

spam()

/Volumes/exFAT_Two/env_datascience/env/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 

 catch me~~ (002)


  after removing the cwd from sys.path.
/Volumes/exFAT_Two/env_datascience/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 

 you got me (003)


  """


<hr>

***The end***.

<hr>